```{contents}
```
## Horizontal Scaling 

### 1. Definition

**Horizontal scaling** (a.k.a. *scale-out*) in Generative AI is the practice of increasing system capacity by **adding more machines or nodes** rather than making individual machines more powerful.

> *Goal:* Support larger models, higher throughput, lower latency, and higher reliability by distributing computation and data across many devices.

---

### 2. Why Horizontal Scaling is Essential in GenAI

Generative AI workloads are characterized by:

| Challenge             | Why Horizontal Scaling Helps          |
| --------------------- | ------------------------------------- |
| Massive models        | Models exceed single-machine memory   |
| High training cost    | Parallelizes compute across GPUs      |
| High inference demand | Serves many users simultaneously      |
| Fault tolerance       | Failures isolated to individual nodes |
| Elastic workloads     | Add/remove nodes based on demand      |

---

### 3. Where Horizontal Scaling is Used

| Layer                | Purpose                                                 |
| -------------------- | ------------------------------------------------------- |
| Training             | Distribute model training across GPUs/nodes             |
| Inference            | Handle large concurrent user requests                   |
| Data processing      | Parallelize dataset loading, tokenization, augmentation |
| Retrieval systems    | Scale vector databases & search                         |
| Monitoring & logging | Scale observability pipelines                           |

---

### 4. Horizontal vs Vertical Scaling

| Feature          | Horizontal Scaling | Vertical Scaling        |
| ---------------- | ------------------ | ----------------------- |
| How              | Add more machines  | Make machines larger    |
| Cost efficiency  | High               | Limited                 |
| Fault tolerance  | Strong             | Weak                    |
| Upper bound      | Very high          | Hardware-limited        |
| Typical in GenAI | **Yes**            | Rarely sufficient alone |

---

### 5. Core Horizontal Scaling Patterns in GenAI

### 5.1 Data Parallelism

Each worker holds a full model copy and processes different data batches.

```
Dataset → split → [GPU1] [GPU2] [GPU3] [GPU4]
```

**Gradient aggregation** keeps models synchronized.

```python
# PyTorch DDP example
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

model = DDP(model)
```

---

### 5.2 Model Parallelism

Split the model itself across devices.

```
[Layers 1-10] → GPU1
[Layers 11-20] → GPU2
```

Used when the model cannot fit on a single device.

---

### 5.3 Pipeline Parallelism

Different micro-batches flow through sequential model partitions.

```
GPU1 → GPU2 → GPU3 → GPU4
```

Improves device utilization for very deep models.

---

### 5.4 Tensor Parallelism

Split large matrix operations across GPUs.

```
Huge Matrix Multiply → shard across GPUs
```

Used heavily in LLM training (Megatron-LM style).

---

### 6. Horizontal Scaling in Training Workflow

```
Data Storage
    ↓
Distributed DataLoader
    ↓
[ Worker Node 1 ] ----\
[ Worker Node 2 ] -----→ Gradient Sync → Parameter Update
[ Worker Node 3 ] ----/
    ↓
Checkpoint Store
```

Key technologies:

* NCCL / Gloo (communication backends)
* Ray, DeepSpeed, Horovod, FSDP, Megatron-LM
* Kubernetes for cluster orchestration

---

### 7. Horizontal Scaling in Inference

**Request-based scaling:**

```
Client Requests → Load Balancer → Inference Nodes (replicas)
```

**Key mechanisms:**

* Auto-scaling based on QPS / latency
* Model sharding across GPUs
* KV-cache partitioning for LLMs
* Batch inference for throughput optimization

---

### 8. Example: Horizontally Scaled Inference with Ray

```python
import ray

@ray.remote(num_gpus=1)
class LLMWorker:
    def generate(self, prompt):
        return model.generate(prompt)

workers = [LLMWorker.remote() for _ in range(8)]

results = ray.get([w.generate.remote("Hello") for w in workers])
```

Adds throughput by simply increasing worker count.

---

### 9. Benefits

* Near-linear scaling of compute
* Supports trillion-parameter models
* Enables global deployment
* High availability & fault tolerance

---

### 10. Practical Limitations

| Bottleneck    | Cause                                |
| ------------- | ------------------------------------ |
| Communication | Gradient & parameter synchronization |
| Network       | Bandwidth & latency                  |
| Memory        | Activations & optimizer states       |
| Stragglers    | Slow nodes reduce overall speed      |

---

### 11. Summary

Horizontal scaling is the **foundation** of modern Generative AI:

* Makes training of massive models feasible
* Enables real-time inference at internet scale
* Provides elasticity, reliability, and performance
* Requires careful orchestration of compute, memory, and communication
